In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install "shapely==2.0.5" trimesh pygltflib


In [2]:
from shapely.geometry import Polygon, box
import trimesh, pygltflib
print("OK:", Polygon, box)


OK: <class 'shapely.geometry.polygon.Polygon'> <function box at 0x00000169E9078680>


In [3]:
# ---- TEMP FIX: stub torchvision.ops.nms so Ultralytics runs without torchvision ----
import sys, types, torch

def _box_iou_one_to_many(b, B):
    # b: (4,) xyxy ; B: (N,4) xyxy  -> IoU (N,)
    x1 = torch.maximum(b[0], B[:, 0])
    y1 = torch.maximum(b[1], B[:, 1])
    x2 = torch.minimum(b[2], B[:, 2])
    y2 = torch.minimum(b[3], B[:, 3])
    inter = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
    area_b  = (b[2] - b[0]).clamp(min=0) * (b[3] - b[1]).clamp(min=0)
    area_B  = (B[:, 2] - B[:, 0]).clamp(min=0) * (B[:, 3] - B[:, 1]).clamp(min=0)
    return inter / (area_b + area_B - inter + 1e-6)

def _nms(boxes, scores, iou_threshold=0.45):
    """
    Minimal NMS replacement: returns indices of kept boxes (LongTensor).
    boxes:  (N,4) xyxy on any device
    scores: (N,)  confidence
    """
    if boxes.numel() == 0:
        return boxes.new_zeros((0,), dtype=torch.long)
    # sort by score desc
    idxs = scores.argsort(descending=True)
    keep = []
    while idxs.numel() > 0:
        i = idxs[0]
        keep.append(i)
        if idxs.numel() == 1:
            break
        ious = _box_iou_one_to_many(boxes[i], boxes[idxs[1:]])
        idxs = idxs[1:][ious <= iou_threshold]
    return torch.stack(keep) if keep else boxes.new_zeros((0,), dtype=torch.long)

# register stub module(s) so `import torchvision` and `from torchvision import ops` succeed
_tv = types.ModuleType("torchvision")
_ops = types.ModuleType("torchvision.ops")
_ops.nms = _nms
_tv.ops = _ops
sys.modules["torchvision"] = _tv
sys.modules["torchvision.ops"] = _ops
# -----------------------------------------------------------------------------


In [4]:
import sys
print(sys.executable)  # verify this is the interpreter you're using

# The easiest, cross-platform triangulator:
!{sys.executable} -m pip install mapbox_earcut

# Optional alternates (either is fine if earcut fails to install):
# !{sys.executable} -m pip install manifold3d
# !{sys.executable} -m pip install triangle


C:\Users\win11\anaconda3\python.exe


In [1]:
# -*- coding: utf-8 -*-
"""
U-Net (walls) + YOLO (openings) -> combined masks -> GLB (3D) for ALL images.

First-time setup in THIS kernel/env:
  pip install shapely==2.0.5 trimesh pygltflib mapbox_earcut

Outputs per image into OUT_DIR:
  *_wall_mask.png, *_openings_mask.png, *_combined_mask.png, *_overlay.png,
  *_vec_debug.png, *_walls.glb
"""

import sys, time
from pathlib import Path
import numpy as np
import cv2
import torch
import torch.nn as nn

# =============== torchvision safety (NMS stub so Ultralytics doesn't require it) ===============
try:
    import torchvision  # noqa
except Exception:
    import types, torch as _torch
    def _nms(boxes, scores, iou_threshold=0.45):
        if boxes.numel() == 0:
            return boxes.new_zeros((0,), dtype=_torch.long)
        idxs = scores.argsort(descending=True)
        keep = []
        while idxs.numel() > 0:
            i = idxs[0]
            keep.append(i)
            if idxs.numel() == 1:
                break
            b  = boxes[i]
            B  = boxes[idxs[1:]]
            x1 = _torch.maximum(b[0], B[:, 0]); y1 = _torch.maximum(b[1], B[:, 1])
            x2 = _torch.minimum(b[2], B[:, 2]); y2 = _torch.minimum(b[3], B[:, 3])
            inter = (x2 - x1).clamp_(min=0) * (y2 - y1).clamp_(min=0)
            area_b = (b[2]-b[0]).clamp_(min=0) * (b[3]-b[1]).clamp_(min=0)
            area_B = (B[:,2]-B[:,0]).clamp_(min=0) * (B[:,3]-B[:,1]).clamp_(min=0)
            ious = inter / (area_b + area_B - inter + 1e-6)
            idxs = idxs[1:][ious <= iou_threshold]
        return _torch.stack(keep) if keep else boxes.new_zeros((0,), dtype=_torch.long)
    _tv = types.ModuleType("torchvision")
    _ops = types.ModuleType("torchvision.ops")
    _ops.nms = _nms
    _tv.ops = _ops
    sys.modules["torchvision"] = _tv
    sys.modules["torchvision.ops"] = _ops
    print("[INFO] Using torchvision.ops.nms stub (torchvision not installed).")

# =============== Ultralytics warmup skip ===============
from ultralytics.nn import autobackend
def _skip_warmup(self, imgsz=(1,3,640,640)):
    return None
autobackend.AutoBackend.warmup = _skip_warmup

from ultralytics import YOLO

# ================== CONFIG ==================
UNET_CKPT    = Path(r"C:\Users\win11\Desktop\Ebni\segmentation\runs_unet_gray_resdrop_noalb\unet_best.pth")
YOLO_WEIGHTS = Path(r"C:\Users\win11\Desktop\Ebni\object detection\runs\detect\ebni-object-detect6\weights\best.pt")  # your YOLO model (COCO for now)
IMAGES_DIR   = Path(r"C:\Users\win11\Desktop\Ebni\Images")
OUT_DIR      = Path(r"C:\Users\win11\Desktop\Ebni\combined_outputs\test_3d")

# U-Net arch (match training)
IMG_SIZE     = 512
IN_CHANNELS  = 1
BASE_CH      = 64
DROPOUT_P    = 0.20
BOTTLENECK_DROPOUT_P = 0.30

# ===== Wall inference controls =====
WALL_BASE_THR   = 0.30           # starting threshold on the probability map
TTA_FLIPS       = True           # average H/V flips
TTA_SCALES      = [1.0, 1.25, 1.5]    # run at 512 and 640 then downscale to 512 and average
TARGET_AREA_MIN = 0.01           # desired wall area ratio range (of 512x512)
TARGET_AREA_MAX = 0.45
ADAPT_STEPS     = 6              # how many small threshold tweaks to try

# ===== Openings class selection =====
# If you have a custom doors/windows YOLO, set these to your numeric class IDs:
FORCE_DOOR_IDS   = None           # e.g., {0}
FORCE_WINDOW_IDS = None          # e.g., {1}
YOLO_IMGSZ       = 1024          # bigger helps floorplans

# Openings handling
DOOR_NAMES   = {"door", "doors"}
WINDOW_NAMES = {"window", "windows"}
OPENING_PAD  = 2
MIN_BOX_AREA = 9

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

assert UNET_CKPT.exists(),    f"UNET_CKPT not found: {UNET_CKPT}"
assert YOLO_WEIGHTS.exists(), f"YOLO_WEIGHTS not found: {YOLO_WEIGHTS}"
assert IMAGES_DIR.exists(),   f"IMAGES_DIR not found: {IMAGES_DIR}"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ================== U-Net (same as training) ==================
class ResDoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2, residual=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.act   = nn.ReLU(inplace=True)
        self.drop  = nn.Dropout2d(p_drop) if p_drop and p_drop>0 else nn.Identity()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.skip  = nn.Conv2d(in_ch, out_ch, 1, bias=False) if in_ch != out_ch else nn.Identity()
        self.residual = residual
    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.drop(out)
        out = self.bn2(self.conv2(out))
        if self.residual:
            identity = self.skip(identity)
            out = out + identity
        out = self.act(out)
        return out

class Down(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2):
        super().__init__()
        self.pool = nn.MaxPool2d(2)
        self.block = ResDoubleConv(in_ch, out_ch, p_drop=p_drop)
    def forward(self, x): return self.block(self.pool(x))

class Up(nn.Module):
    def __init__(self, in_ch, out_ch, p_drop=0.2):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
        self.block = ResDoubleConv(in_ch, out_ch, p_drop=p_drop)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        dy = x2.size(2) - x1.size(2); dx = x2.size(3) - x1.size(3)
        x1 = nn.functional.pad(x1, [dx//2, dx-dx//2, dy//2, dy-dy//2])
        return self.block(torch.cat([x2, x1], dim=1))

class UNetResDrop(nn.Module):
    def __init__(self, in_channels=1, num_classes=1, base_ch=64, p_drop=0.2, p_drop_bot=0.3):
        super().__init__()
        self.inc   = ResDoubleConv(in_channels, base_ch, p_drop=p_drop)
        self.down1 = Down(base_ch, base_ch*2, p_drop=p_drop)
        self.down2 = Down(base_ch*2, base_ch*4, p_drop=p_drop)
        self.down3 = Down(base_ch*4, base_ch*8, p_drop=p_drop)
        self.down4 = Down(base_ch*8, base_ch*16, p_drop=p_drop_bot)
        self.up1   = Up(base_ch*16+base_ch*8, base_ch*8, p_drop=p_drop)
        self.up2   = Up(base_ch*8+base_ch*4,  base_ch*4, p_drop=p_drop)
        self.up3   = Up(base_ch*4+base_ch*2,  base_ch*2, p_drop=p_drop)
        self.up4   = Up(base_ch*2+base_ch,    base_ch,   p_drop=p_drop)
        self.outc  = nn.Conv2d(base_ch, 1, 1)
    def forward(self, x):
        x1 = self.inc(x); x2 = self.down1(x1); x3 = self.down2(x2); x4 = self.down3(x3); x5 = self.down4(x4)
        x  = self.up1(x5, x4); x = self.up2(x, x3); x = self.up3(x, x2); x = self.up4(x, x1)
        return self.outc(x)

def load_unet(ckpt_path: Path):
    ckpt = torch.load(str(ckpt_path), map_location="cpu")
    model = UNetResDrop(in_channels=IN_CHANNELS, num_classes=1,
                        base_ch=BASE_CH, p_drop=DROPOUT_P, p_drop_bot=BOTTLENECK_DROPOUT_P)
    state = ckpt["model"] if "model" in ckpt else ckpt
    model.load_state_dict(state, strict=True)
    model.to(DEVICE).eval()
    return model

def _prep_gray(img_path: Path):
    img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(img_path)
    if img.ndim == 2:
        g = img
    elif img.ndim == 3:
        if img.shape[2] == 1:   g = img[:, :, 0]
        elif img.shape[2] == 3: g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif img.shape[2] == 4: g = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        else:                   g = np.squeeze(img)
    else:
        g = np.squeeze(img)
    if g.ndim != 2:
        raise ValueError(f"Expected 2D grayscale after conversion, got {g.shape}")
    return g

def _run_unet_single(model, gray_2d, out_size):
    """Run once at given size; returns probability map in [0,1] at out_size."""
    if gray_2d.shape[:2] != (out_size, out_size):
        ximg = cv2.resize(gray_2d, (out_size, out_size), interpolation=cv2.INTER_LINEAR)
    else:
        ximg = gray_2d
    x = ((ximg.astype(np.float32) / 255.0) - 0.5) / 0.5
    x = torch.from_numpy(x).unsqueeze(0).unsqueeze(0).to(DEVICE)    # [1,1,H,W]
    with torch.no_grad():
        p = torch.sigmoid(model(x))[0, 0].cpu().numpy()
    return p

def infer_wall_mask(model, img_path: Path, img_size=IMG_SIZE):
    gray = _prep_gray(img_path)
    H0, W0 = gray.shape[:2]

    # ------- TTA: scales + flips -------
    acc = np.zeros((img_size, img_size), np.float32); n = 0
    scales = TTA_SCALES if TTA_SCALES else [1.0]
    flips  = [(False, False)]
    if TTA_FLIPS:
        flips = [(False, False), (True, False), (False, True), (True, True)]

    for s in scales:
        in_size = int(round(img_size * s))
        for fh, fv in flips:
            g = gray
            if fh: g = cv2.flip(g, 1)
            if fv: g = cv2.flip(g, 0)
            prob = _run_unet_single(model, g, in_size)
            # undo flips, bring back to img_size
            if fh: prob = np.flip(prob, axis=1)
            if fv: prob = np.flip(prob, axis=0)
            if prob.shape != (img_size, img_size):
                prob = cv2.resize(prob, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
            acc += prob; n += 1

    prob = acc / max(n, 1)

    # ------- adaptive threshold to reach plausible area ratio -------
    thr = WALL_BASE_THR
    mask = (prob > thr).astype(np.uint8)
    area = mask.sum() / float(mask.size)
    step = 0.06  # tweak step
    for _ in range(ADAPT_STEPS):
        if area < TARGET_AREA_MIN:
            thr = max(0.05, thr - step)
        elif area > TARGET_AREA_MAX:
            thr = min(0.95, thr + step)
        else:
            break
        mask = (prob > thr).astype(np.uint8)
        area = mask.sum() / float(mask.size)
        step *= 0.6  # smaller nudges

    # save debug probability heatmap & threshold text for visibility
    stem = Path(img_path).stem
    heat = (np.clip(prob, 0, 1) * 255).astype(np.uint8)
    cv2.imwrite(str(OUT_DIR / f"{stem}_prob.png"), heat)
    with open((OUT_DIR / f"{stem}_thr.txt"), "w") as f:
        f.write(f"thr={thr:.3f} area={area:.4f} n_tta={n}\n")

    # to uint8 0/255
    mask_u8 = (mask * 255).astype(np.uint8)
    return mask_u8, (H0, W0)


# ================== YOLO openings -> mask ==================
def yolo_openings_mask(yolo_model, img_path: Path, mask_shape):
    Hm, Wm = mask_shape
    img_bgr = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(img_path)
    H0, W0 = img_bgr.shape[:2]
    sx, sy = Wm / W0, Hm / H0

    res = yolo_model(str(img_path), imgsz=YOLO_IMGSZ)[0]

    if FORCE_DOOR_IDS is not None or FORCE_WINDOW_IDS is not None:
        door_ids   = set(FORCE_DOOR_IDS or [])
        window_ids = set(FORCE_WINDOW_IDS or [])
        names = getattr(yolo_model, "names", {})
    else:
        names = {i: n.lower() for i, n in getattr(yolo_model, "names", {}).items()}
        door_ids   = {i for i,n in names.items() if n in DOOR_NAMES}
        window_ids = {i for i,n in names.items() if n in WINDOW_NAMES}

    openings = np.zeros((Hm, Wm), np.uint8)
    kept = 0
    for b in res.boxes:
        cls = int(b.cls[0].item())
        if (cls not in door_ids) and (cls not in window_ids):
            continue
        x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())
        X1 = int(max(0, x1 * sx - OPENING_PAD))
        Y1 = int(max(0, y1 * sy - OPENING_PAD))
        X2 = int(min(Wm, x2 * sx + OPENING_PAD))
        Y2 = int(min(Hm, y2 * sy + OPENING_PAD))
        if (X2 - X1) * (Y2 - Y1) < MIN_BOX_AREA:
            continue
        cv2.rectangle(openings, (X1, Y1), (X2, Y2), 255, -1)
        kept += 1

    if kept == 0:
        print(f"[YOLO] {Path(img_path).name}: no openings detected. names={names}")
    return openings


# ================== Morphology helpers to refine walls ==================
def _remove_small_islands(bin01: np.ndarray, min_area=64):
    num, lab, stats, _ = cv2.connectedComponentsWithStats(bin01, connectivity=8)
    out = np.zeros_like(bin01)
    for i in range(1, num):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            out[lab == i] = 1
    return out

def _fill_small_holes(bin01: np.ndarray, min_area=64):
    h, w = bin01.shape
    inv = (1 - bin01).astype(np.uint8)
    num, lab, stats, _ = cv2.connectedComponentsWithStats(inv, connectivity=8)
    out = bin01.copy()
    for i in range(1, num):
        x, y, ww, hh, area = stats[i]
        touches_border = (x == 0 or y == 0 or x + ww == w or y + hh == h)
        if (not touches_border) and area <= min_area:
            out[lab == i] = 1
    return out

def refine_walls(mask_u8: np.uint8, close_px=3, min_island_area=128, min_hole_area=128):
    m = (mask_u8 > 0).astype(np.uint8)
    if close_px > 0:
        k = cv2.getStructuringElement(cv2.MORPH_RECT, (close_px*2+1, close_px*2+1))
        m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, k, 1)
    m = _remove_small_islands(m, min_area=min_island_area)
    m = _fill_small_holes(m,   min_area=min_hole_area)
    return (m * 255).astype(np.uint8)

# ================== Vectorization & triangulation to GLB (Polygon-only pipeline) ==================
from shapely.geometry import Polygon, MultiPolygon
from shapely.validation import make_valid
from shapely.geometry.polygon import orient as shapely_orient
from shapely.ops import unary_union, triangulate as shp_triangulate
import shapely.affinity as affinity
import trimesh
from trimesh.creation import extrude_triangulation

PIXEL_TO_METER  = 0.01   # 1 px = 1 cm
WALL_HEIGHT_M   = 3.0
SIMPLIFY_EPS    = 0.0    # keep 0 while stabilizing
MIN_AREA_PX     = 2
DILATE_PX       = 4      # thicken thin walls before contours

# Try earcut (best). If not present we’ll fall back to Delaunay per polygon.
try:
    import mapbox_earcut as earcut
    _HAS_EARCUT = True
    _EARCUT_UINT32  = hasattr(earcut, "triangulate_uint32")
    _EARCUT_FLOAT64 = hasattr(earcut, "triangulate_float64")
except Exception:
    _HAS_EARCUT = False
    _EARCUT_UINT32 = _EARCUT_FLOAT64 = False

def _mask_to_polygons_cv(bin_mask: np.ndarray,
                         min_area_px=2,
                         simplify_eps=0.0) -> list[Polygon]:
    """
    0/255 mask -> list of Polygon (with holes), via OpenCV contour hierarchy.
    No MultiLineString / GeometryCollection outputs.
    """
    m = (bin_mask > 0).astype(np.uint8)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((3,3),np.uint8), 1)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN,  np.ones((3,3),np.uint8), 1)

    contours, hierarchy = cv2.findContours(m, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    if hierarchy is None or not contours:
        return []

    hierarchy = hierarchy[0]  # [next, prev, first_child, parent]
    children = {}; outers = []
    for i,h in enumerate(hierarchy):
        if h[3] == -1: outers.append(i)
        else: children.setdefault(h[3], []).append(i)

    polys = []
    for oi in outers:
        ext = contours[oi].squeeze(1)
        if ext.ndim != 2 or ext.shape[0] < 3: continue
        ext_poly = make_valid(Polygon(ext))
        if ext_poly.is_empty: continue

        holes = []
        for hi in children.get(oi, []):
            hc = contours[hi].squeeze(1)
            if hc.ndim != 2 or hc.shape[0] < 3: continue
            hp = make_valid(Polygon(hc))
            if isinstance(hp, Polygon) and (not hp.is_empty) and hp.area >= min_area_px:
                holes.append(list(hp.exterior.coords))

        poly = make_valid(Polygon(ext_poly.exterior.coords, holes))
        parts = list(poly.geoms) if isinstance(poly, MultiPolygon) else [poly]
        for p in parts:
            if not isinstance(p, Polygon) or p.is_empty or p.area < min_area_px: continue
            if simplify_eps > 0.0:
                p = p.simplify(simplify_eps, preserve_topology=True)
                if p.is_empty or p.area < min_area_px: continue
            polys.append(p)

    return polys


try:
    import mapbox_earcut as earcut
    _HAS_EARCUT = True
    _EARCUT_UINT32  = hasattr(earcut, "triangulate_uint32")
    _EARCUT_FLOAT64 = hasattr(earcut, "triangulate_float64")
except Exception:
    _HAS_EARCUT = False
    _EARCUT_UINT32 = _EARCUT_FLOAT64 = False

def _earcut_triangulate(p_m: Polygon):
    def ring(coords):
        pts = list(coords)
        return pts[:-1] if len(pts)>1 and pts[0]==pts[-1] else pts
    ext  = ring(p_m.exterior.coords)
    holes = [ring(r.coords) for r in p_m.interiors]
    if len(ext) < 3: raise ValueError("exterior ring too small")

    verts = []; hole_idx=[]
    for x,y in ext: verts += [x,y]
    off = len(ext)
    for h in holes:
        hole_idx.append(off)
        for x,y in h: verts += [x,y]
        off += len(h)

    v = np.array(verts, dtype=np.float64)
    if _EARCUT_UINT32:  idx = earcut.triangulate_uint32(v, hole_idx, 2)
    elif _EARCUT_FLOAT64: idx = earcut.triangulate_float64(v, hole_idx, 2).astype(np.uint32)
    else: raise RuntimeError("earcut API not present")

    V = np.array(ext + [xy for h in holes for xy in h], dtype=np.float64)
    F = idx.reshape(-1,3).astype(np.int64)
    return V, F

def _delaunay_triangulate(p_m: Polygon):
    tris = shp_triangulate(p_m)
    verts=[]; faces=[]; vmap={}
    def vid(xy):
        if xy not in vmap:
            vmap[xy] = len(verts); verts.append(xy)
        return vmap[xy]
    for t in tris:
        t_clip = t.intersection(p_m)
        if t_clip.is_empty: continue
        coords = list(t_clip.exterior.coords)
        if len(coords) < 4: continue
        pts=[]
        for (x,y) in coords[:-1]:
            if len(pts)==3: break
            if not pts or (x,y)!=pts[-1]: pts.append((float(x),float(y)))
        if len(pts)!=3: continue
        faces.append((vid(pts[0]), vid(pts[1]), vid(pts[2])))
    if not faces: raise ValueError("Delaunay produced no faces")
    return np.array(verts, np.float64), np.array(faces, np.int64)


# Fallback: raster -> 3D voxels -> marching cubes -> mesh
def export_glb_voxel(mask_u8: np.ndarray, out_glb: Path,
                     px_to_m: float = 0.01, height_m: float = 3.0,
                     layers: int = 16, downsample: int = 2) -> bool:
    """
    Build a 3D mesh from the raster mask by stacking it into a voxel volume.
    No triangulation. Always returns a mesh (may be heavier).
    """
    from trimesh.voxel import VoxelGrid
    m = (mask_u8 > 0).astype(np.uint8)
    if downsample > 1:
        h, w = m.shape
        m = cv2.resize(m, (w//downsample, h//downsample), interpolation=cv2.INTER_NEAREST)

    # volume: (Z,Y,X) -> layers copies along height
    vol = np.repeat(m[None, :, :], layers, axis=0).astype(bool)
    vg = VoxelGrid(vol)
    mesh = vg.marching_cubes  # iso-surface

    # scale: X,Z = px_to_m ; Y (height) = height_m / layers
    S = np.diag([px_to_m*downsample, height_m/layers, px_to_m*downsample, 1.0])
    mesh.apply_transform(S)
    out_glb.parent.mkdir(parents=True, exist_ok=True)
    mesh.export(str(out_glb))
    print(f"[OK] 3D walls (voxel) saved -> {out_glb}")
    return True



def export_glb_from_mask(combined_mask: np.ndarray, out_glb: Path, debug_png: Path|None=None) -> bool:
    # A) binarize + thicken + refine (solid walls)
    m = combined_mask
    if m.dtype != np.uint8: m = (m>0).astype(np.uint8)*255
    if DILATE_PX>0:
        k = cv2.getStructuringElement(cv2.MORPH_RECT, (DILATE_PX*2+1, DILATE_PX*2+1))
        m = cv2.dilate(m, k, 1)
    m = refine_walls(m, close_px=2, min_island_area=32, min_hole_area=32)

    # B) contours -> list[Polygon] ONLY (no collections)
    polys = _mask_to_polygons_cv(m, min_area_px=MIN_AREA_PX, simplify_eps=SIMPLIFY_EPS)
    polys = [p for p in polys if isinstance(p, Polygon) and (not p.is_empty) and (p.area >= MIN_AREA_PX)]

    # optional debug
    if debug_png is not None:
        dbg = cv2.cvtColor(m, cv2.COLOR_GRAY2BGR)
        for p in polys:
            pts = np.array(p.exterior.coords, np.int32).reshape(-1,1,2)
            cv2.polylines(dbg, [pts], True, (0,255,0), 2)
            for r in p.interiors:
                pts = np.array(r.coords, np.int32).reshape(-1,1,2)
                cv2.polylines(dbg, [pts], True, (0,165,255), 2)
        cv2.imwrite(str(debug_png), dbg)

    # C) Triangulation path (Delaunay-only in safe mode)
    meshes = []
    if polys:
        print(f"[GLB] polygons to extrude (polygons-only): {len(polys)}")
        for p in polys:
            p_or = shapely_orient(p, sign=1.0).buffer(0)
            p_m  = affinity.scale(p_or, xfact=PIXEL_TO_METER, yfact=PIXEL_TO_METER, origin=(0,0)).buffer(0)
            try:
                V, F = _delaunay_triangulate(p_m)  # earcut disabled in safe mode
            except Exception as e:
                print("[WARN] triangulation failed:", e)
                continue
            if V.shape[0]==0 or F.shape[0]==0:
                continue
            try:
                meshes.append(extrude_triangulation(V, F, WALL_HEIGHT_M))
            except Exception as e:
                print("[WARN] extrusion failed:", e)
                continue
        if meshes:
            out_glb.parent.mkdir(parents=True, exist_ok=True)
            trimesh.Scene(meshes).export(str(out_glb))
            print(f"[OK] 3D walls saved -> {out_glb}")
            return True

    # D) Last resort: **voxel** (never triangulates; very stable)
    if VOXEL_FALLBACK:
        print("[GLB] Triangulation produced nothing — using voxel fallback.")
        return export_glb_voxel(m, out_glb, px_to_m=PIXEL_TO_METER, height_m=WALL_HEIGHT_M,
                                layers=VOXEL_LAYERS, downsample=VOXEL_DOWNSAMPLE)
    else:
        print("[WARN] no meshes produced and voxel fallback disabled.")
        return False








# ================== Combine + save one ==================
def combine_and_save(unet_model, yolo_model, img_path: Path):
    wall_mask, (H0, W0) = infer_wall_mask(unet_model, img_path, IMG_SIZE)
    openings_mask = yolo_openings_mask(yolo_model, img_path, wall_mask.shape)

    # combine then refine for solid walls
    combined = cv2.bitwise_and(wall_mask, cv2.bitwise_not(openings_mask))
    combined = refine_walls(combined, close_px=3, min_island_area=128, min_hole_area=128)

    stem = img_path.stem
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(OUT_DIR / f"{stem}_wall_mask.png"),      wall_mask)
    cv2.imwrite(str(OUT_DIR / f"{stem}_openings_mask.png"),  openings_mask)
    cv2.imwrite(str(OUT_DIR / f"{stem}_combined_mask.png"),  combined)

    vis = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if vis is not None:
        comb_up = cv2.resize(combined, (vis.shape[1], vis.shape[0]), interpolation=cv2.INTER_NEAREST)
        edges = cv2.Canny(comb_up, 50, 150) > 0
        vis[edges] = (0,255,0)
        cv2.imwrite(str(OUT_DIR / f"{stem}_overlay.png"), vis)

    out_glb = OUT_DIR / f"{stem}_walls.glb"
    dbg     = OUT_DIR / f"{stem}_vec_debug.png"
    ok = export_glb_from_mask(combined, out_glb, debug_png=dbg)
    if not ok:
        print("[GLB] Fallback: trying refined wall_mask only")
        walls_ref = refine_walls(wall_mask, close_px=2, min_island_area=32, min_hole_area=32)
        ok2 = export_glb_from_mask(walls_ref, out_glb, debug_png=OUT_DIR / f"{stem}_vec_debug_walls_only.png")
        print(f"[GLB] Fallback result: {ok2}")
        return ok2
    return ok

# ================== MAIN (batch) ==================
if __name__ == "__main__":
    unet = load_unet(UNET_CKPT)
    yolo = YOLO(str(YOLO_WEIGHTS))

    IMG_EXTS = {".png",".jpg",".jpeg",".bmp",".tif",".tiff"}
    files = [p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in IMG_EXTS]
    assert files, f"No images under {IMAGES_DIR}"

    print(f"[BATCH] Processing {len(files)} images from: {IMAGES_DIR}")
    ok = glb_ok = failed = 0
    t0 = time.time()
    for p in files:
        try:
            res = combine_and_save(unet, yolo, p)
            ok += 1; glb_ok += int(bool(res))
        except Exception as e:
            failed += 1; print(f"[WARN] failed on {p.name}: {e}")
    dt = time.time() - t0

    print("\n========== SUMMARY ==========")
    print(f"Total images     : {len(files)}")
    print(f"OK (PNG saved)   : {ok}")
    print(f"GLB exported     : {glb_ok}")
    print(f"Failed           : {failed}")
    print(f"Outputs at       : {OUT_DIR.resolve()}")
    print(f"Elapsed          : {dt:.1f}s")
    print("================================")


[INFO] Using torchvision.ops.nms stub (torchvision not installed).


C:\Users\win11\AppData\Local\Temp\ipykernel_5412\4003781788.py:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(str(ckpt_path), map_location="cpu")


[BATCH] Processing 14 images from: C:\Users\win11\Desktop\Ebni\Images

image 1/1 C:\Users\win11\Desktop\Ebni\Images\IMG-20250721-WA0008_jpg.rf.05fb32a79edf315c366f5f75e6f3d00d.jpg: 1024x1024 3 doors, 9 windows, 366.3ms
Speed: 14.1ms preprocess, 366.3ms inference, 4.9ms postprocess per image at shape (1, 3, 1024, 1024)
[GLB] polygons to extrude (polygons-only): 10
[WARN] triangulation failed: 'MultiLineString' object has no attribute 'exterior'
[WARN] triangulation failed: 'MultiLineString' object has no attribute 'exterior'
[WARN] triangulation failed: 'MultiLineString' object has no attribute 'exterior'
[WARN] triangulation failed: 'GeometryCollection' object has no attribute 'exterior'
[WARN] triangulation failed: 'GeometryCollection' object has no attribute 'exterior'
[WARN] triangulation failed: 'GeometryCollection' object has no attribute 'exterior'
[WARN] triangulation failed: 'GeometryCollection' object has no attribute 'exterior'
[WARN] triangulation failed: 'GeometryCollection